# Setup

In [ ]:
import pickle
import random
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

import _config as cfg  # noqa: E402
import DataSets_validation_CONCAT as ds  # noqa: E402
import kaplanmeier as km  # noqa: E402
import matplotlib.pyplot as plt  # noqa: E402
import numpy as np  # noqa: E402
import pandas as pd  # noqa: E402
from imblearn.over_sampling import SMOTE  # noqa: E402
from sklearnex import patch_sklearn  # noqa: E402

patch_sklearn()

from sklearn import svm  # noqa: E402
from sklearn.metrics import (  # noqa: E402
    accuracy_score,
    auc,
    balanced_accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from tqdm.notebook import tqdm  # noqa: E402

yeloh_seed = 2137

random.seed(1024)
np.random.seed(1024)

results_dir = Path("Results")
results_dir.mkdir(parents=True, exist_ok=True)

validation_dir = results_dir / "validation"
validation_dir.mkdir(parents=True, exist_ok=True)
validation_clinical_features_file = "/data/teamgdansk/katy-variants/validation-datasets/KATY_Edin_cohort_clinical_features.csv"


def svm_model_file(model_name, cut_input_params=False):
    model_name = model_name.replace(" ", "_")
    if cut_input_params:
        name_suffix = "-cut"
    else:
        name_suffix = "-full"
    return results_dir / f"svm-model-{model_name}{name_suffix}.pkl"


# Matplotlib config
plt.rcdefaults()

main_font_size = 6
label_font_size = 10

axis_color = "0.15"

plt.rcParams["text.color"] = axis_color
plt.rcParams["axes.labelcolor"] = axis_color
plt.rcParams["xtick.color"] = axis_color
plt.rcParams["ytick.color"] = axis_color
plt.rcParams["axes.edgecolor"] = axis_color

plt.rcParams["font.size"] = main_font_size
plt.rcParams["axes.labelsize"] = main_font_size
plt.rcParams["axes.titlesize"] = main_font_size
plt.rcParams["xtick.labelsize"] = main_font_size
plt.rcParams["ytick.labelsize"] = main_font_size
plt.rcParams["legend.fontsize"] = main_font_size
plt.rcParams["figure.labelsize"] = main_font_size
plt.rcParams["figure.titlesize"] = main_font_size

plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

plt.rcParams["savefig.transparent"] = True

plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["ps.fonttype"] = 42
plt.rcParams["svg.fonttype"] = "none"


def figsize_in_mm(width, height):
    mm = 1 / 25.4  # millimeters in inches
    return (width * mm, height * mm)

# Training and Testing

In [ ]:
def svm_train_test(train_data, train_y, test_data, verbose=False, clf_kwargs={}):
    sm = SMOTE(random_state=432)
    train_data, train_y = sm.fit_resample(train_data, train_y)
    clf = svm.SVC(probability=True, random_state=yeloh_seed, **clf_kwargs)
    clf = clf.fit(train_data, train_y)

    predictions = clf.predict(test_data)
    probs = clf.predict_proba(test_data)
    probSV = [i[1] for i in probs]
    if verbose:
        print(probSV)
    new_pd = pd.DataFrame(probSV)
    return clf, predictions, new_pd

In [ ]:
# Config

# Set `True` if you want to use only age, gender, and mutation data:
cut_input_params = True

# Figures config
legend_font_size = 5
text_font_size = 5

fig_roc_size = figsize_in_mm(48, 37)

fig_surv_size = figsize_in_mm(56, 36)

fig_sc_size = figsize_in_mm(100, 40)
fig_sc_bar_width = 0.2

#

if cut_input_params:
    name_suffix = "-cut"
else:
    name_suffix = "-full"

mut_vec_len_label = "mut_vec_len"
clf_params_label = "clf_params"
model_name_label = "plot_label"

m_num = len(cfg.configurations)

model_colors = {n[model_name_label]: f"C{i}" for i, n in enumerate(cfg.configurations)}
model_colors_surv_lines = {}
i = 0
for n in cfg.configurations:
    lines_c = [plt.color_sequences["tab20"][i + 1], plt.color_sequences["tab20"][i]]
    model_colors_surv_lines[n[model_name_label]] = lines_c
    i += 2

fig_roc = plt.figure(figsize=fig_roc_size, layout="constrained")
ax_roc = fig_roc.add_subplot()

test_scores_str = ""
test_scores_names = ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC"]
test_scores = {}

surv_ylabel = True

for config in tqdm(cfg.configurations, leave=False):
    dimension_of_embedding_vectors = config[mut_vec_len_label]
    clf_params = config[clf_params_label]
    model_name = config[model_name_label]

    train_data, train_y, test_data, test_y, test_pfs, _, _, test_pfs_cnsr, _, _, _ = (
        ds.transforming_Braun_dataset(
            config,
            dimension_of_embedding_vectors=dimension_of_embedding_vectors,
            cut_input_params=cut_input_params,
        )
    )

    model, svm_linear_preds, svm_prob = svm_train_test(
        train_data, train_y, test_data, clf_kwargs=clf_params
    )

    probs = svm_prob.values.flatten()
    fpr, tpr, _ = roc_curve(test_y, probs)
    roc_auc = auc(fpr, tpr)

    prec = precision_score(test_y, svm_linear_preds)
    rec = recall_score(test_y, svm_linear_preds)
    f1 = f1_score(test_y, svm_linear_preds)
    acc = accuracy_score(test_y, svm_linear_preds)
    rauc = roc_auc_score(test_y, probs)

    ba = balanced_accuracy_score(test_y, svm_linear_preds)

    test_scores_str += f"{model_name}:\n"
    test_scores_str += f"  {test_scores_names[0]}: {acc:.3f}\n"
    test_scores_str += f"  {test_scores_names[1]}: {prec:.3f}\n"
    test_scores_str += f"  {test_scores_names[2]}: {rec:.3f}\n"
    test_scores_str += f"  {test_scores_names[3]}: {f1:.3f}\n"
    test_scores_str += f"  {test_scores_names[4]}: {rauc:.3f}\n"
    test_scores_str += f"    _MEAN_: {(acc + prec + rec + f1 + rauc) / 5:.3f}\n"
    test_scores_str += f"    BA: {ba:.3f}\n"

    test_scores[model_name] = {}
    test_scores[model_name][test_scores_names[0]] = acc
    test_scores[model_name][test_scores_names[1]] = prec
    test_scores[model_name][test_scores_names[2]] = rec
    test_scores[model_name][test_scores_names[3]] = f1
    test_scores[model_name][test_scores_names[4]] = rauc

    _ = ax_roc.plot(
        fpr,
        tpr,
        label=f"{model_name} (AUC={roc_auc:.3f})",
        color=model_colors[model_name],
        linewidth=1,
    )

    with open(svm_model_file(model_name, cut_input_params), "wb") as f:
        pickle.dump(model, f)

    # Response prediction
    time_event = test_pfs.reset_index(drop=True)
    censoring = test_pfs_cnsr.reset_index(drop=True)
    # censoring = pd.Series([1 if p == -1 else 0 for p in svm_linear_preds])
    labx = pd.Series(
        ["Responders" if y == 1 else "Non-responders" for y in svm_linear_preds]
    )
    # labx = pd.Series(["Responders" if y == 1 else "Non-responders" for y in test_y])
    surv_pred = km.fit(time_event, censoring, labx)
    logrank_p = surv_pred["logrank_P"]
    fig_surv, ax_surv = km.plot(
        surv_pred,
        figsize=fig_surv_size,
        fontsize=main_font_size,
        cmap=model_colors_surv_lines[model_name],
    )
    # old_title = ax_surv.get_title()
    # p_value_str = old_title.replace("Survival function\n", "")
    ax_surv.set_title(None)
    ax_surv.text(
        0.85,
        0.3,
        f"Logrank P-value:\n{logrank_p:.3e}",
        ha="right",
        va="bottom",
        transform=ax_surv.transAxes,
        fontsize=text_font_size,
    )
    ax_surv.set_title(f"{model_name}")
    # ax_surv.set_title(
    #     old_title.replace("Survival function", f"{model_name}"),
    #     {"fontsize": label_font_size},
    # )
    ax_surv.tick_params(length=3.5)
    ax_surv.set_xlabel("Time (months)")
    if surv_ylabel:
        ax_surv.set_ylabel("Probability of response")
        surv_ylabel = False
    ax_surv.legend(
        handlelength=0.8,
        handletextpad=0.5,
        labelspacing=0.8,
        fontsize=legend_font_size,
        loc="upper right",
        # loc="upper center",
        # bbox_to_anchor=(0.5, -0.4),
    )
    for fmt in ["pdf", "png"]:
        fig_surv.savefig(
            results_dir.joinpath(
                f"response-prediction-{model_name.replace(' ', '_')}{name_suffix}.{fmt}"
            ),
            bbox_inches="tight",
            pad_inches=0.03,
            dpi=600,
        )

# ROC
ax_roc.set_xlabel("False Positive Rate")
ax_roc.set_ylabel("True Positive Rate")
# ax_roc.set_title("ROC Curves for different levels", {"fontsize": label_font_size})
ax_roc.legend(loc="lower right", fontsize=legend_font_size)
for fmt in ["pdf", "png"]:
    fig_roc.savefig(results_dir.joinpath(f"roc-curves{name_suffix}.{fmt}"), dpi=600)

# Print and save scores to a file
print(test_scores_str)
with open(results_dir.joinpath(f"scores{name_suffix}.txt"), "w") as f:
    f.write(test_scores_str)

# Scores figure
m_names = list(test_scores.keys())
s_num = len(test_scores_names)
x = np.arange(s_num)
b_pos = np.arange(
    fig_sc_bar_width / 2 * (1 - m_num),
    fig_sc_bar_width / 2 * (m_num - 1),
    fig_sc_bar_width,
)
fig_sc = plt.figure(figsize=fig_sc_size, layout="constrained")
ax_sc = fig_sc.add_subplot()
for n, p in zip(m_names, b_pos):
    _ = ax_sc.bar(
        x + p,
        list(test_scores[n].values()),
        width=fig_sc_bar_width,
        label=n,
        color=model_colors[n],
        zorder=10,
    )
ax_sc.set_xticks(x, test_scores_names)
ax_sc.grid(axis="y")
_ = ax_sc.legend(
    fontsize=legend_font_size, ncols=m_num, loc="lower right", bbox_to_anchor=(1, 1)
)
for fmt in ["pdf", "png"]:
    fig_sc.savefig(results_dir.joinpath(f"scores{name_suffix}.{fmt}"), dpi=600)

# Validation

In [ ]:
# Config

# Set `True` if you want to use only age, gender, and mutation data:
cut_input_params = True

# Figures config
legend_font_size = 5
text_font_size = 5

fig_roc_size = figsize_in_mm(48, 37)

fig_surv_size = figsize_in_mm(56, 36)

fig_sc_size = figsize_in_mm(100, 40)
fig_sc_bar_width = 0.2

#

if cut_input_params:
    name_suffix = "-cut"
else:
    name_suffix = "-full"

mut_vec_len_label = "mut_vec_len"
clf_params_label = "clf_params"
model_name_label = "plot_label"

m_num = len(cfg.configurations)

model_colors = {n[model_name_label]: f"C{i}" for i, n in enumerate(cfg.configurations)}
model_colors_surv_lines = {}
i = 0
for n in cfg.configurations:
    lines_c = [plt.color_sequences["tab20"][i + 1], plt.color_sequences["tab20"][i]]
    model_colors_surv_lines[n[model_name_label]] = lines_c
    i += 2

fig_roc = plt.figure(figsize=fig_roc_size, layout="constrained")
ax_roc = fig_roc.add_subplot()

test_scores_str = ""
test_scores_names = ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC"]
test_scores = {}

surv_ylabel = True

#
d = []
#

for config in tqdm(cfg.configurations, leave=False):
    dimension_of_embedding_vectors = config[mut_vec_len_label]
    clf_params = config[clf_params_label]
    model_name = config[model_name_label]

    #
    # config["with_mutations"] = False
    #

    train_data, train_y, test_data, test_y, test_pfs, _, _, _, val_data, val_y, val_pfs = ds.transforming_Braun_dataset(
        config,
        dimension_of_embedding_vectors=dimension_of_embedding_vectors,
        cut_input_params=cut_input_params,
        validation_features_file=validation_clinical_features_file,
    )

    #
    d.append([train_data, train_y, test_data, test_y, test_pfs, val_data, val_y, val_pfs])
    #

    input_data = test_data
    input_y = test_y
    input_pfs = test_pfs

    # Load SVM model and run validation data

    clf_file = svm_model_file(model_name, cut_input_params)
    with open(clf_file, "rb") as f:
        clf = pickle.load(f)

    svm_linear_preds = clf.predict(input_data)
    probs = clf.predict_proba(input_data)
    probSV = [i[1] for i in probs]
    svm_prob = pd.DataFrame(probSV)

    probs = svm_prob.values.flatten()
    fpr, tpr, _ = roc_curve(input_y, probs)
    roc_auc = auc(fpr, tpr)

    prec = precision_score(input_y, svm_linear_preds)
    rec = recall_score(input_y, svm_linear_preds)
    f1 = f1_score(input_y, svm_linear_preds)
    acc = accuracy_score(input_y, svm_linear_preds)
    rauc = roc_auc_score(input_y, probs)

    ba = balanced_accuracy_score(input_y, svm_linear_preds)

    test_scores_str += f"{model_name}:\n"
    test_scores_str += f"  {test_scores_names[0]}: {acc:.3f}\n"
    test_scores_str += f"  {test_scores_names[1]}: {prec:.3f}\n"
    test_scores_str += f"  {test_scores_names[2]}: {rec:.3f}\n"
    test_scores_str += f"  {test_scores_names[3]}: {f1:.3f}\n"
    test_scores_str += f"  {test_scores_names[4]}: {rauc:.3f}\n"
    test_scores_str += f"    _MEAN_: {(acc + prec + rec + f1 + rauc) / 5:.3f}\n"
    test_scores_str += f"    BA: {ba:.3f}\n"

    test_scores[model_name] = {}
    test_scores[model_name][test_scores_names[0]] = acc
    test_scores[model_name][test_scores_names[1]] = prec
    test_scores[model_name][test_scores_names[2]] = rec
    test_scores[model_name][test_scores_names[3]] = f1
    test_scores[model_name][test_scores_names[4]] = rauc

    _ = ax_roc.plot(
        fpr,
        tpr,
        label=f"{model_name} (AUC={roc_auc:.3f})",
        color=model_colors[model_name],
        linewidth=1,
    )

    # Response prediction
    time_event = input_pfs.reset_index(drop=True)
    # True only for this particular case
    val_pfs_cnsr = pd.Series([0] * input_data.shape[0])
    #
    censoring = val_pfs_cnsr.reset_index(drop=True)
    # censoring = pd.Series([1 if p == -1 else 0 for p in svm_linear_preds])
    labx = pd.Series(
        ["Responders" if y == 1 else "Non-responders" for y in svm_linear_preds]
    )
    # labx = pd.Series(["Responders" if y == 1 else "Non-responders" for y in test_y])
    surv_pred = km.fit(time_event, censoring, labx)
    logrank_p = surv_pred["logrank_P"]
    fig_surv, ax_surv = km.plot(
        surv_pred,
        figsize=fig_surv_size,
        fontsize=main_font_size,
        cmap=model_colors_surv_lines[model_name],
    )
    # old_title = ax_surv.get_title()
    # p_value_str = old_title.replace("Survival function\n", "")
    ax_surv.set_title(None)
    ax_surv.text(
        0.85,
        0.3,
        f"Logrank P-value:\n{logrank_p:.3e}",
        ha="right",
        va="bottom",
        transform=ax_surv.transAxes,
        fontsize=text_font_size,
    )
    ax_surv.set_title(f"{model_name}")
    # ax_surv.set_title(
    #     old_title.replace("Survival function", f"{model_name}"),
    #     {"fontsize": label_font_size},
    # )
    ax_surv.tick_params(length=3.5)
    ax_surv.set_xlabel("Time (months)")
    if surv_ylabel:
        ax_surv.set_ylabel("Probability of response")
        surv_ylabel = False
    ax_surv.legend(
        handlelength=0.8,
        handletextpad=0.5,
        labelspacing=0.8,
        fontsize=legend_font_size,
        loc="upper right",
        # loc="upper center",
        # bbox_to_anchor=(0.5, -0.4),
    )
    for fmt in ["pdf", "png"]:
        fig_surv.savefig(
            validation_dir.joinpath(
                f"val-response-prediction-{model_name.replace(' ', '_')}{name_suffix}.{fmt}"
            ),
            bbox_inches="tight",
            pad_inches=0.03,
            dpi=600,
        )

# ROC
ax_roc.set_xlabel("False Positive Rate")
ax_roc.set_ylabel("True Positive Rate")
# ax_roc.set_title("ROC Curves for different levels", {"fontsize": label_font_size})
ax_roc.legend(loc="lower right", fontsize=legend_font_size)
for fmt in ["pdf", "png"]:
    fig_roc.savefig(
        validation_dir.joinpath(f"val-roc-curves{name_suffix}.{fmt}"), dpi=600
    )

# Print and save scores to a file
print(test_scores_str)
with open(validation_dir.joinpath(f"val-scores{name_suffix}.txt"), "w") as f:
    f.write(test_scores_str)

# Scores figure
m_names = list(test_scores.keys())
s_num = len(test_scores_names)
x = np.arange(s_num)
b_pos = np.arange(
    fig_sc_bar_width / 2 * (1 - m_num),
    fig_sc_bar_width / 2 * (m_num - 1),
    fig_sc_bar_width,
)
fig_sc = plt.figure(figsize=fig_sc_size, layout="constrained")
ax_sc = fig_sc.add_subplot()
for n, p in zip(m_names, b_pos):
    _ = ax_sc.bar(
        x + p,
        list(test_scores[n].values()),
        width=fig_sc_bar_width,
        label=n,
        color=model_colors[n],
        zorder=10,
    )
ax_sc.set_xticks(x, test_scores_names)
ax_sc.grid(axis="y")
_ = ax_sc.legend(
    fontsize=legend_font_size, ncols=m_num, loc="lower right", bbox_to_anchor=(1, 1)
)
for fmt in ["pdf", "png"]:
    fig_sc.savefig(validation_dir.joinpath(f"val-scores{name_suffix}.{fmt}"), dpi=600)

In [ ]:
org_cf = pd.read_csv("../data/Braun_2020_ALL_UNIQUE_final_reduced.csv")

In [ ]:
d[0][0]

In [ ]:
d[0][0]

In [ ]:
d[1][0]

In [ ]:
d[1][0]

In [ ]:
d[2][0]

In [ ]:
d[2][0]

In [ ]:
d[0][5].loc[d[0][9].isin(d[0][8])]

In [ ]:
d[0][0].equals(d[0][5].loc[d[0][9].isin(d[0][8])])

In [ ]:
d[0][8]

In [ ]:
d[0][9].loc[d[0][9].isin(d[0][8])]

In [ ]:
d[0][9].loc[org_cf["TrainTestStatus"] == "Train"]